In [1]:
import pandas as pd
import requests
import json
import time
import grequests
import numpy as np
pd.set_option('display.max_columns', None) 

C:\Users\Varsha\Anaconda3\lib\site-packages\grequests.py:21: MonkeyPatchWarning: Monkey-patching ssl after ssl has already been imported may lead to errors, including RecursionError on Python 3.6. It may also silently lead to incorrect behaviour on Python 3.7. Please monkey-patch earlier. See https://github.com/gevent/gevent/issues/1016. Modules that had direct imports (NOT patched): ['urllib3.contrib.pyopenssl (C:\\Users\\Varsha\\Anaconda3\\lib\\site-packages\\urllib3\\contrib\\pyopenssl.py)', 'urllib3.util (C:\\Users\\Varsha\\Anaconda3\\lib\\site-packages\\urllib3\\util\\__init__.py)']. 
  curious_george.patch_all(thread=False, select=False)
C:\Users\Varsha\Anaconda3\lib\site-packages\gevent\hub.py:154: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [2]:
movie_data = pd.read_csv("Movies_Data_Updated.csv")
movie_data = movie_data.drop('Unnamed: 0', axis=1)
movie_data.head()

,Close,Movie Title,O Theaters,Open,Opening,Rank,Studio,TG Theaters,Total Gross,Year,Title,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,Language,Country,Awards,Poster,Ratings,Metascore,imdbRating,imdbVotes,imdbID,DVD,BoxOffice,Production,Website
0,NaN,Vice Squad,510.0,22-Jan,"$1,717,205",1,Emb,510.0,"$13,253,583",1982,Vice Squad,R,22-Jan-82,97 min,"Action, Crime, Drama",Gary Sherman,"Sandy Howard, Kenneth Peters, Robert Vincent O...","Season Hubley, Gary Swanson, Wings Hauser, Pep...","A Los Angeles businesswoman, known only by her...",English,"UK, USA",NaN,https://m.media-amazon.com/images/M/MV5BMTRhYj...,"[{'Source': 'Internet Movie Database', 'Value'...",NaN,6.4,"1,469",tt0084861,21-Feb-06,NaN,Anchor Bay Entertainment,NaN
1,NaN,The Seduction,597.0,22-Jan,"$1,503,543",2,Emb,597.0,"$11,378,339",1982,The Seduction,R,29-Jan-82,104 min,Thriller,David Schmoeller,David Schmoeller,"Morgan Fairchild, Michael Sarrazin, Vince Edwa...",Los Angeles anchorwoman Jaime Douglas has it a...,English,USA,3 nominations.,https://m.media-amazon.com/images/M/MV5BZDljY2...,"[{'Source': 'Internet Movie Database', 'Value'...",NaN,4.9,799,tt0084652,7-Nov-06,NaN,Embassy,NaN
2,NaN,Shoot the Moon,585.0,22-Jan,"$1,870,000",3,MGM,585.0,"$9,217,530",1982,Shoot the Moon,R,19-Feb-82,124 min,Drama,Alan Parker,Bo Goldman,"Albert Finney, Diane Keaton, Karen Allen, Pete...","A fifteen year marriage dissolves, leaving bot...",English,USA,Nominated for 2 Golden Globes. Another 5 nomin...,https://m.media-amazon.com/images/M/MV5BZTk2ZW...,"[{'Source': 'Internet Movie Database', 'Value'...",NaN,7.0,"2,167",tt0084675,6-Nov-07,NaN,Warner Home Video,NaN
3,NaN,The Border,84.0,29-Jan,"$516,014",4,Uni.,888.0,"$6,118,683",1982,The Border,R,12-Feb-82,108 min,"Crime, Drama",Tony Richardson,"Deric Washburn, Walon Green, David Freeman","Jack Nicholson, Harvey Keitel, Valerie Perrine...",U.S. Border Patrol agent Charlie Smith just wa...,"English, Spanish",USA,NaN,https://m.media-amazon.com/images/M/MV5BYmNiMz...,"[{'Source': 'Internet Movie Database', 'Value'...",NaN,6.4,"5,316",tt0083678,11-May-04,NaN,Universal Pictures,NaN
4,NaN,Venom,900.0,29-Jan,"$1,672,747",5,Par.,900.0,"$5,229,643",1982,Venom,PG-13,5-Oct-18,112 min,"Action, Sci-Fi, Thriller",Ruben Fleischer,"Jeff Pinkner (screenplay by), Scott Rosenberg ...","Tom Hardy, Michelle Williams, Riz Ahmed, Scott...",A failed reporter is bonded to an alien entity...,"English, Mandarin, Malay","China, USA",NaN,https://m.media-amazon.com/images/M/MV5BNzAwNz...,"[{'Source': 'Internet Movie Database', 'Value'...",35.0,6.8,"257,961",tt1270797,18-Jun-13,NaN,Vis,NaN


In [3]:
movie_data = movie_data.drop(movie_data.loc[movie_data["Movie Title"] != movie_data["Title"], :].index)
unclean_data = movie_data.loc[movie_data["Movie Title"] != movie_data["Title"], :]
unclean_data

,Close,Movie Title,O Theaters,Open,Opening,Rank,Studio,TG Theaters,Total Gross,Year,Title,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,Language,Country,Awards,Poster,Ratings,Metascore,imdbRating,imdbVotes,imdbID,DVD,BoxOffice,Production,Website


In [4]:
dups = pd.concat(g for _, g in movie_data.groupby("Movie Title") if len(g) > 1)
dups

,Close,Movie Title,O Theaters,Open,Opening,Rank,Studio,TG Theaters,Total Gross,Year,Title,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,Language,Country,Awards,Poster,Ratings,Metascore,imdbRating,imdbVotes,imdbID,DVD,BoxOffice,Production,Website
5351,4-Mar,11-Sep,1.0,18-Jul,"$7,046",29,Emp.,6.0,"$127,035",2003,11-Sep,Unrated,11-Sep-02,134 min,Drama,"Youssef Chahine, Amos Gitai, Shôhei Imamura, A...","Alain Brigand (original idea), Youssef Chahine...","Maryam Karimi, Mohamad Dolati, Agelem Habibi, ...",Eleven directors from 11 countries each contri...,"English, Spanish, French, Arabic, Hebrew, Pers...","UK, France, Egypt, Japan, Mexico, USA, Iran",3 wins & 1 nomination.,https://m.media-amazon.com/images/M/MV5BMzkxMj...,"[{'Source': 'Internet Movie Database', 'Value'...",61.0,7.0,"5,206",tt0328802,26-Oct-04,NaN,Empire Pictures,http://www.empirepicturesusa.com/ep/show_movie...
13606,10-Sep,11-Sep,425.0,8-Sep,"$170,000",37,ADC,425.0,"$170,000",2017,11-Sep,TV-PG,10-Mar-02,112 min,Documentary,"James Hanlon, Gédéon Naudet, Jules Naudet","Tom Forman, Greg Kandra","Tony Benatatos, Jamal Braithwaite, Steve Busce...","On September 11, 2001, filmmaker James Hanlon ...",English,"USA, France",Won 2 Primetime Emmys. Another 6 wins & 6 nomi...,https://m.media-amazon.com/images/M/MV5BMTIyMz...,"[{'Source': 'Internet Movie Database', 'Value'...",NaN,8.4,"6,005",tt0312318,12-Sep-02,NaN,Goldfish Pictures,NaN
8871,20-May,Bluebeard,1.0,26-Mar,"$8,370",32,Strand,3.0,"$33,490",2010,Bluebeard,APPROVED,11-Nov-44,72 min,"Crime, Film-Noir, Horror",Edgar G. Ulmer,"Arnold Lipp (story), Werner H. Furst (story), ...","John Carradine, Jean Parker, Nils Asther, Ludw...","A killer of young women, dubbed Bluebeard, is ...",English,USA,NaN,https://images-na.ssl-images-amazon.com/images...,"[{'Source': 'Internet Movie Database', 'Value'...",NaN,6.0,"1,358",tt0036653,29-Jun-04,NaN,PRC Pictures Inc.,NaN
13242,30-Mar,Bluebeard,14.0,17-Mar,"$21,234",40,WGUSA,14.0,"$43,144",2017,Bluebeard,APPROVED,11-Nov-44,72 min,"Crime, Film-Noir, Horror",Edgar G. Ulmer,"Arnold Lipp (story), Werner H. Furst (story), ...","John Carradine, Jean Parker, Nils Asther, Ludw...","A killer of young women, dubbed Bluebeard, is ...",English,USA,NaN,https://images-na.ssl-images-amazon.com/images...,"[{'Source': 'Internet Movie Database', 'Value'...",NaN,6.0,"1,358",tt0036653,29-Jun-04,NaN,PRC Pictures Inc.,NaN
5113,11-Sep,Chaos,3.0,29-Jan,"$20,570",20,NYer,5.0,"$206,789",2003,Chaos,R,15-Dec-05,106 min,"Action, Crime, Drama, Thriller",Tony Giglio,Tony Giglio,"Jason Statham, Ryan Phillippe, Wesley Snipes, ...","In Seattle, detective Quentin Conners is unfai...",English,"Canada, UK, USA",NaN,https://m.media-amazon.com/images/M/MV5BMTc3ND...,"[{'Source': 'Internet Movie Database', 'Value'...",NaN,6.5,"48,528",tt0402910,19-Feb-08,NaN,Rogue Pictures,http://www.tfmdistribution.fr/fiches_films/cha...
6406,-,Chaos,NaN,10-Aug,NaN,43,Din.,2.0,"$10,289",2005,Chaos,R,15-Dec-05,106 min,"Action, Crime, Drama, Thriller",Tony Giglio,Tony Giglio,"Jason Statham, Ryan Phillippe, Wesley Snipes, ...","In Seattle, detective Quentin Conners is unfai...",English,"Canada, UK, USA",NaN,https://m.media-amazon.com/images/M/MV5BMTc3ND...,"[{'Source': 'Internet Movie Database', 'Value'...",NaN,6.5,"48,528",tt0402910,19-Feb-08,NaN,Rogue Pictures,http://www.tfmdistribution.fr/fiches_films/cha...
5223,31-Jul,City of Ghosts,6.0,25-Apr,"$30,547",24,UA,20.0,"$357,197",2003,City of Ghosts,R,27-Mar-03,116 min,"Crime, Drama, Thriller",Matt Dillon,"Matt Dillon, Barry Gifford","Matt Dillon, James Caan, Natascha McElhone, Gé...",A con man flees to Southeast Asia when an inte...,"English, French, Khmer, Russian",USA,NaN,https://m.media-amazon.com/images/M/MV5BMTE5OD...,"[{'Source': 'Internet Movie Database', 'Value'...",57.0,6.0,"4,688",tt0164003,28-Oct-03,"$275,998",MGM Pictures,http://www.cityofghostsmovie.com
13477,7-Sep,City of Ghosts,2.0,7-Jul,"$13,601",23,IFC,18.0,"$128,015",2017,City of Ghosts,R,27-Mar-03,116 min,"Crime

In [5]:
def appendData(reqs):
    for index, row in enumerate(dups.itertuples()):
            try:
                item = reqs[index].json()
                movie_data.at[row.Index, "Title"] = item['Title'] 
                movie_data.at[row.Index, "Rated"] = item['Rated'] 
                movie_data.at[row.Index, "Released"] = item['Released'] 
                movie_data.at[row.Index, "Runtime"] = item['Runtime'] 
                movie_data.at[row.Index, "Genre"] = item['Genre'] 
                movie_data.at[row.Index, "Director"] = item['Director'] 
                movie_data.at[row.Index, "Writer"] = item['Writer'] 
                movie_data.at[row.Index, "Actors"] = item['Actors'] 
                movie_data.at[row.Index, "Plot"] = item['Plot'] 
                movie_data.at[row.Index, "Language"] = item['Language'] 
                movie_data.at[row.Index, "Country"] = item['Country'] 
                movie_data.at[row.Index, "Awards"] = item['Awards'] 
                movie_data.at[row.Index, "Poster"] = item['Poster'] 
                movie_data.at[row.Index, "Ratings"] = item['Ratings'] 
                movie_data.at[row.Index, "Metascore"] = item['Metascore'] 
                movie_data.at[row.Index, "imdbRating"] = item['imdbRating'] 
                movie_data.at[row.Index, "imdbVotes"] = item['imdbVotes'] 
                movie_data.at[row.Index, "imdbID"] = item['imdbID'] 
                movie_data.at[row.Index, "DVD"] = item['DVD'] 
                movie_data.at[row.Index, "BoxOffice"] = item['BoxOffice'] 
                movie_data.at[row.Index, "Production"] = item['Production'] 
                movie_data.at[row.Index, "Website"] = item['Website'] 

            except: 
                print("Record not found")

In [6]:
def exception_handler(request, exception):
    print("Request failed")

urls = ["http://www.omdbapi.com/?t={0}&y={1}&plot=full&apikey={2}".format(row[2].replace(" ", "+"), row[10], "504e6438") for row in dups.itertuples()]
rs = (grequests.get(u) for u in urls)
requests = grequests.map(rs, exception_handler=exception_handler, size=10)
appendData(requests)

Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found


In [7]:
movie_data = movie_data.drop(movie_data.loc[movie_data["Movie Title"] != movie_data["Title"], :].index)
movie_data = movie_data.reset_index(drop=True)
movie_data

,Close,Movie Title,O Theaters,Open,Opening,Rank,Studio,TG Theaters,Total Gross,Year,Title,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,Language,Country,Awards,Poster,Ratings,Metascore,imdbRating,imdbVotes,imdbID,DVD,BoxOffice,Production,Website
0,NaN,Vice Squad,510.0,22-Jan,"$1,717,205",1,Emb,510.0,"$13,253,583",1982,Vice Squad,R,22-Jan-82,97 min,"Action, Crime, Drama",Gary Sherman,"Sandy Howard, Kenneth Peters, Robert Vincent O...","Season Hubley, Gary Swanson, Wings Hauser, Pep...","A Los Angeles businesswoman, known only by her...",English,"UK, USA",NaN,https://m.media-amazon.com/images/M/MV5BMTRhYj...,"[{'Source': 'Internet Movie Database', 'Value'...",NaN,6.4,"1,469",tt0084861,21-Feb-06,NaN,Anchor Bay Entertainment,NaN
1,NaN,The Seduction,597.0,22-Jan,"$1,503,543",2,Emb,597.0,"$11,378,339",1982,The Seduction,R,29-Jan-82,104 min,Thriller,David Schmoeller,David Schmoeller,"Morgan Fairchild, Michael Sarrazin, Vince Edwa...",Los Angeles anchorwoman Jaime Douglas has it a...,English,USA,3 nominations.,https://m.media-amazon.com/images/M/MV5BZDljY2...,"[{'Source': 'Internet Movie Database', 'Value'...",NaN,4.9,799,tt0084652,7-Nov-06,NaN,Embassy,NaN
2,NaN,Shoot the Moon,585.0,22-Jan,"$1,870,000",3,MGM,585.0,"$9,217,530",1982,Shoot the Moon,R,19-Feb-82,124 min,Drama,Alan Parker,Bo Goldman,"Albert Finney, Diane Keaton, Karen Allen, Pete...","A fifteen year marriage dissolves, leaving bot...",English,USA,Nominated for 2 Golden Globes. Another 5 nomin...,https://m.media-amazon.com/images/M/MV5BZTk2ZW...,"[{'Source': 'Internet Movie Database', 'Value'...",NaN,7.0,"2,167",tt0084675,6-Nov-07,NaN,Warner Home Video,NaN
3,NaN,The Border,84.0,29-Jan,"$516,014",4,Uni.,888.0,"$6,118,683",1982,The Border,R,12-Feb-82,108 min,"Crime, Drama",Tony Richardson,"Deric Washburn, Walon Green, David Freeman","Jack Nicholson, Harvey Keitel, Valerie Perrine...",U.S. Border Patrol agent Charlie Smith just wa...,"English, Spanish",USA,NaN,https://m.media-amazon.com/images/M/MV5BYmNiMz...,"[{'Source': 'Internet Movie Database', 'Value'...",NaN,6.4,"5,316",tt0083678,11-May-04,NaN,Universal Pictures,NaN
4,NaN,Venom,900.0,29-Jan,"$1,672,747",5,Par.,900.0,"$5,229,643",1982,Venom,PG-13,5-Oct-18,112 min,"Action, Sci-Fi, Thriller",Ruben Fleischer,"Jeff Pinkner (screenplay by), Scott Rosenberg ...","Tom Hardy, Michelle Williams, Riz Ahmed, Scott...",A failed reporter is bonded to an alien entity...,"English, Mandarin, Malay","China, USA",NaN,https://m.media-amazon.com/images/M/MV5BNzAwNz...,"[{'Source': 'Internet Movie Database', 'Value'...",35.0,6.8,"257,961",tt1270797,18-Jun-13,NaN,Vis,NaN
5,NaN,"I, the Jury",NaN,9-Jan,NaN,7,Fox,NaN,"$1,515,578",1982,"I, the Jury",R,22-Apr-82,111 min,"Crime, Drama, Mystery, Romance, Thriller",Richard T. Heffron,"Mickey Spillane (novel), Larry Cohen (screenplay)","Armand Assante, Barbara Carrera, Laurene Lando...",An old friend of a private detective is murder...,English,USA,NaN,https://m.media-amazon.com/images/M/MV5BMmJjY2...,"[{'Source': 'Internet Movie Database', 'Value'...",NaN,5.9,"1,258",tt0084112,26-Jul-90,NaN,Fox,NaN
6,NaN,Quest for Fire,223.0,12-Feb,"$2,171,303",1,Fox,426.0,"$20,959,585",1982,Quest for Fire,R,12-Feb-82,100 min,"Adventure, Drama, History",Jean-Jacques Annaud,"Gérard Brach (screenplay), J.H. Rosny Sr. (bas...","Everett McGill, Ron Perlman, Nicholas Kadi, Ra...",Anthony Burgess created the primitive language...,"None, French","Canada, France",Won 1 Oscar. Another 10 wins & 7 nominations.,https://m.media-amazon.com/images/M/MV5BNzIwMG...,"[{'Source': 'Internet Movie Database', 'Value'...",79.0,7.4,"18,752",tt0082484,4-Mar-03,NaN,Fox,NaN
7,NaN,Death Wish II,377.0,19-Feb,"$1,871,487",2,Film,377.0,"$16,100,000",1982,Death Wish II,R,19-Feb-82,89 min,"Action, Crime, Drama, Thriller",Michael Winner,"David Engelbach, Brian Garfield (characters)","Charles Bronson, Jill Ireland, Vincent Gardeni...","Paul Kersey, the vigilante, now lives in LA wi...",English,USA,2 nominations.,https://m.media-amazon.com/im

In [8]:
movie_data["Actors"] = [actors.split(",") if pd.notna(actors) else float('nan') for actors in movie_data["Actors"]]
movie_data["Genre"] = [[genre.strip() for genre in genres.split(",")] if pd.notna(genres) else float('nan') for genres in movie_data["Genre"]]                                   
movie_data

,Close,Movie Title,O Theaters,Open,Opening,Rank,Studio,TG Theaters,Total Gross,Year,Title,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,Language,Country,Awards,Poster,Ratings,Metascore,imdbRating,imdbVotes,imdbID,DVD,BoxOffice,Production,Website
0,NaN,Vice Squad,510.0,22-Jan,"$1,717,205",1,Emb,510.0,"$13,253,583",1982,Vice Squad,R,22-Jan-82,97 min,"[Action, Crime, Drama]",Gary Sherman,"Sandy Howard, Kenneth Peters, Robert Vincent O...","[Season Hubley, Gary Swanson, Wings Hauser, ...","A Los Angeles businesswoman, known only by her...",English,"UK, USA",NaN,https://m.media-amazon.com/images/M/MV5BMTRhYj...,"[{'Source': 'Internet Movie Database', 'Value'...",NaN,6.4,"1,469",tt0084861,21-Feb-06,NaN,Anchor Bay Entertainment,NaN
1,NaN,The Seduction,597.0,22-Jan,"$1,503,543",2,Emb,597.0,"$11,378,339",1982,The Seduction,R,29-Jan-82,104 min,[Thriller],David Schmoeller,David Schmoeller,"[Morgan Fairchild, Michael Sarrazin, Vince E...",Los Angeles anchorwoman Jaime Douglas has it a...,English,USA,3 nominations.,https://m.media-amazon.com/images/M/MV5BZDljY2...,"[{'Source': 'Internet Movie Database', 'Value'...",NaN,4.9,799,tt0084652,7-Nov-06,NaN,Embassy,NaN
2,NaN,Shoot the Moon,585.0,22-Jan,"$1,870,000",3,MGM,585.0,"$9,217,530",1982,Shoot the Moon,R,19-Feb-82,124 min,[Drama],Alan Parker,Bo Goldman,"[Albert Finney, Diane Keaton, Karen Allen, ...","A fifteen year marriage dissolves, leaving bot...",English,USA,Nominated for 2 Golden Globes. Another 5 nomin...,https://m.media-amazon.com/images/M/MV5BZTk2ZW...,"[{'Source': 'Internet Movie Database', 'Value'...",NaN,7.0,"2,167",tt0084675,6-Nov-07,NaN,Warner Home Video,NaN
3,NaN,The Border,84.0,29-Jan,"$516,014",4,Uni.,888.0,"$6,118,683",1982,The Border,R,12-Feb-82,108 min,"[Crime, Drama]",Tony Richardson,"Deric Washburn, Walon Green, David Freeman","[Jack Nicholson, Harvey Keitel, Valerie Perr...",U.S. Border Patrol agent Charlie Smith just wa...,"English, Spanish",USA,NaN,https://m.media-amazon.com/images/M/MV5BYmNiMz...,"[{'Source': 'Internet Movie Database', 'Value'...",NaN,6.4,"5,316",tt0083678,11-May-04,NaN,Universal Pictures,NaN
4,NaN,Venom,900.0,29-Jan,"$1,672,747",5,Par.,900.0,"$5,229,643",1982,Venom,PG-13,5-Oct-18,112 min,"[Action, Sci-Fi, Thriller]",Ruben Fleischer,"Jeff Pinkner (screenplay by), Scott Rosenberg ...","[Tom Hardy, Michelle Williams, Riz Ahmed, S...",A failed reporter is bonded to an alien entity...,"English, Mandarin, Malay","China, USA",NaN,https://m.media-amazon.com/images/M/MV5BNzAwNz...,"[{'Source': 'Internet Movie Database', 'Value'...",35.0,6.8,"257,961",tt1270797,18-Jun-13,NaN,Vis,NaN
5,NaN,"I, the Jury",NaN,9-Jan,NaN,7,Fox,NaN,"$1,515,578",1982,"I, the Jury",R,22-Apr-82,111 min,"[Crime, Drama, Mystery, Romance, Thriller]",Richard T. Heffron,"Mickey Spillane (novel), Larry Cohen (screenplay)","[Armand Assante, Barbara Carrera, Laurene La...",An old friend of a private detective is murder...,English,USA,NaN,https://m.media-amazon.com/images/M/MV5BMmJjY2...,"[{'Source': 'Internet Movie Database', 'Value'...",NaN,5.9,"1,258",tt0084112,26-Jul-90,NaN,Fox,NaN
6,NaN,Quest for Fire,223.0,12-Feb,"$2,171,303",1,Fox,426.0,"$20,959,585",1982,Quest for Fire,R,12-Feb-82,100 min,"[Adventure, Drama, History]",Jean-Jacques Annaud,"Gérard Brach (screenplay), J.H. Rosny Sr. (bas...","[Everett McGill, Ron Perlman, Nicholas Kadi,...",Anthony Burgess created the primitive language...,"None, French","Canada, France",Won 1 Oscar. Another 10 wins & 7 nominations.,https://m.media-amazon.com/images/M/MV5BNzIwMG...,"[{'Source': 'Internet Movie Database', 'Value'...",79.0,7.4,"18,752",tt0082484,4-Mar-03,NaN,Fox,NaN
7,NaN,Death Wish II,377.0,19-Feb,"$1,871,487",2,Film,377.0,"$16,100,000",1982,Death Wish II,R,19-Feb-82,89 min,"[Action, Crime, Drama, Thriller]",Michael Winner,"David Engelbach, Brian Garfield (characters)","[Charles Bronson, Jill Ireland, Vincent Gard...","Paul Kersey, the vigilante, now lives in LA wi...",English,USA,2 nominations.,https://m.media-amazon.com/imag

In [9]:
genres = ["Action", "Adventure", "Animation", "Biography", "Comedy", "Crime", 
            "Documentary", "Drama", "Family", "Fantasy", "Film Noir", "History", "Horror",
            "Music", "Musical", "Mystery", "Romance", "Sci-Fi", "Short", "Sport", "Superhero", 
            "Thriller", "War", "Western"]    
    
for movie in movie_data.itertuples():
    try:
        for genre in genres:
            if genre in movie[15]:
                movie_data.at[movie.Index, genre] = 1
            else: 
                movie_data.at[movie.Index, genre] = 0
    except:
        pass
    
movie_data

,Close,Movie Title,O Theaters,Open,Opening,Rank,Studio,TG Theaters,Total Gross,Year,Title,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,Language,Country,Awards,Poster,Ratings,Metascore,imdbRating,imdbVotes,imdbID,DVD,BoxOffice,Production,Website,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,Film Noir,History,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Short,Sport,Superhero,Thriller,War,Western
0,NaN,Vice Squad,510.0,22-Jan,"$1,717,205",1,Emb,510.0,"$13,253,583",1982,Vice Squad,R,22-Jan-82,97 min,"[Action, Crime, Drama]",Gary Sherman,"Sandy Howard, Kenneth Peters, Robert Vincent O...","[Season Hubley, Gary Swanson, Wings Hauser, ...","A Los Angeles businesswoman, known only by her...",English,"UK, USA",NaN,https://m.media-amazon.com/images/M/MV5BMTRhYj...,"[{'Source': 'Internet Movie Database', 'Value'...",NaN,6.4,"1,469",tt0084861,21-Feb-06,NaN,Anchor Bay Entertainment,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,NaN,The Seduction,597.0,22-Jan,"$1,503,543",2,Emb,597.0,"$11,378,339",1982,The Seduction,R,29-Jan-82,104 min,[Thriller],David Schmoeller,David Schmoeller,"[Morgan Fairchild, Michael Sarrazin, Vince E...",Los Angeles anchorwoman Jaime Douglas has it a...,English,USA,3 nominations.,https://m.media-amazon.com/images/M/MV5BZDljY2...,"[{'Source': 'Internet Movie Database', 'Value'...",NaN,4.9,799,tt0084652,7-Nov-06,NaN,Embassy,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,NaN,Shoot the Moon,585.0,22-Jan,"$1,870,000",3,MGM,585.0,"$9,217,530",1982,Shoot the Moon,R,19-Feb-82,124 min,[Drama],Alan Parker,Bo Goldman,"[Albert Finney, Diane Keaton, Karen Allen, ...","A fifteen year marriage dissolves, leaving bot...",English,USA,Nominated for 2 Golden Globes. Another 5 nomin...,https://m.media-amazon.com/images/M/MV5BZTk2ZW...,"[{'Source': 'Internet Movie Database', 'Value'...",NaN,7.0,"2,167",tt0084675,6-Nov-07,NaN,Warner Home Video,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,NaN,The Border,84.0,29-Jan,"$516,014",4,Uni.,888.0,"$6,118,683",1982,The Border,R,12-Feb-82,108 min,"[Crime, Drama]",Tony Richardson,"Deric Washburn, Walon Green, David Freeman","[Jack Nicholson, Harvey Keitel, Valerie Perr...",U.S. Border Patrol agent Charlie Smith just wa...,"English, Spanish",USA,NaN,https://m.media-amazon.com/images/M/MV5BYmNiMz...,"[{'Source': 'Internet Movie Database', 'Value'...",NaN,6.4,"5,316",tt0083678,11-May-04,NaN,Universal Pictures,NaN,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,NaN,Venom,900.0,29-Jan,"$1,672,747",5,Par.,900.0,"$5,229,643",1982,Venom,PG-13,5-Oct-18,112 min,"[Action, Sci-Fi, Thriller]",Ruben Fleischer,"Jeff Pinkner (screenplay by), Scott Rosenberg ...","[Tom Hardy, Michelle Williams, Riz Ahmed, S...",A failed reporter is bonded to an alien entity...,"English, Mandarin, Malay","China, USA",NaN,https://m.media-amazon.com/images/M/MV5BNzAwNz...,"[{'Source': 'Internet Movie Database', 'Value'...",35.0,6.8,"257,961",tt1270797,18-Jun-13,NaN,Vis,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
5,NaN,"I, the Jury",NaN,9-Jan,NaN,7,Fox,NaN,"$1,515,578",1982,"I, the Jury",R,22-Apr-82,111 min,"[Crime, Drama, Mystery, Romance, Thriller]",Richard T. Heffron,"Mickey Spillane (novel), Larry Cohen (screenplay)","[Armand Assante, Barbara Carrera, Laurene La...",An old friend of a private detective is murder...,English,USA,NaN,https://m.media-amazon.com/images/M/MV5BMmJjY2...,"[{'Source': 'Internet Movie Database', 'Value'...",NaN,5.9,"1,258",tt0084112,26-Jul-90,NaN,Fox,NaN,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
6,NaN,Quest for Fire,223.0,12-Feb,"$2,171,303",1,Fox,426.0,"$20,959,585",1982,Quest for Fire,R,12-Feb-82,100 min,"[Adventure, Drama, History]",Jean-